# 1. Load Data

In [1]:
# Install Packages
#conda install ipywidgets

In [1]:
# Load data
import os
import pandas as pd
import glob
import json
import numpy as np
from sklearn.neighbors import BallTree
from tqdm.notebook import tqdm
import gc
import ipywidgets
from joblib import Parallel, delayed


# Confirm the current working directory
print("Current working directory:", os.getcwd())

Current working directory: /home/qusta100/Gasoline


In [3]:
# Path to the price data folder
folder_path = "Data/Prices/"

# Find all CSV files matching the pattern "*-prices.csv"
csv_files = glob.glob(os.path.join(folder_path, "*-prices.csv"))

# List to store daily data
df_list = []

# Read each file and add it to the list
for file in csv_files:
    # Read: timestamp, UUID, Diesel, E5, E10
    df_day = pd.read_csv(file, usecols=[0, 1, 2, 3, 4])
    df_day.columns = ["date", "station_uuid", "diesel", "e5", "e10"]
    df_list.append(df_day)

# Combine all daily data into a single DataFrame
df = pd.concat(df_list, ignore_index=True)

# Ensure correct column names (optional)
df.columns = ["date", "station_uuid", "diesel", "e5", "e10"]

# Parse timestamps and remove timezone
df["date"] = pd.to_datetime(df["date"], utc=True).dt.tz_convert(None)

# Sort by station and time
df = df.sort_values(["station_uuid", "date"])

# Get the overall time range (start & end)
start = df["date"].min().replace(minute=0, second=0)
end = df["date"].max().replace(minute=45, second=0)

# Create 15-minute time intervals
quarter_hours = pd.date_range(start=start, end=end, freq="15min")


In [4]:
# Load all station files from March and April
station_files = glob.glob("Data/Stations/2025-0[3-4]-*-stations.csv")

stations_list = []
for file in station_files:
    df_station = pd.read_csv(file)
    # Extract the date from the filename and convert to datetime
    date_str = os.path.basename(file).split("-stations.csv")[0]
    df_station["day"] = pd.to_datetime(date_str)
    stations_list.append(df_station)

# Combine all station data into a single DataFrame
stations_all = pd.concat(stations_list, ignore_index=True)

# Keep only the latest entry per station (based on the 'day' column)
stations_latest = stations_all.sort_values("day").drop_duplicates("uuid", keep="last")

In [5]:
# Merge price data with the latest station information
df = df.merge(
    stations_latest,
    how="left",
    left_on="station_uuid",
    right_on="uuid"
)

# 2. Process Data

In [6]:
# Result list
resampled_list = []

# List of fuel price columns
price_cols = ["diesel", "e5", "e10"]

# List of metadata columns (everything except date, UUID, and prices)
metadata_cols = [col for col in df.columns if col not in ["date", "station_uuid"] + price_cols]

# For each station: resample time series and attach metadata
for station_id, group in df.groupby("station_uuid"):
    group = group.set_index("date").sort_index()

    # Resample prices (forward fill) for each fuel type
    reindexed_prices = {
        col: group[col].reindex(quarter_hours, method="ffill")
        for col in price_cols
    }

    # DataFrame with timestamps and UUID
    result = pd.DataFrame({
        "date": quarter_hours,
        "station_uuid": station_id,
        **{col: reindexed_prices[col].values for col in price_cols}
    })

    # Metadata from the first row (if available)
    meta = group[metadata_cols].iloc[0] if not group.empty else pd.Series(index=metadata_cols)

    # Attach metadata to each row
    for col in metadata_cols:
        result[col] = meta[col]

    resampled_list.append(result)

# Merge all resampled data
resampled_df = pd.concat(resampled_list, ignore_index=True)

# Restrict to April
resampled_df = resampled_df[resampled_df["date"] >= pd.Timestamp("2025-04-01 00:00:00")]


In [7]:
# Remove rows with missing metadata
df_clean = resampled_df[resampled_df[["latitude", "longitude"]].notna().all(axis=1)].copy()

# Remove rows with all prices missing
df_clean = df_clean[df_clean[["diesel", "e5", "e10"]].notna().any(axis=1)].copy()

In [8]:
df_clean.head(5)

,date,station_uuid,diesel,e5,e10,uuid,name,brand,street,house_number,post_code,city,latitude,longitude,first_active,openingtimes_json,day
2980,2025-04-01 00:00:00,00006210-0037-4444-8888-acdc00006210,1.649,1.779,1.719,00006210-0037-4444-8888-acdc00006210,Beducker - Qualität günstig tanken,Beducker,Donauwörther Str.,47a,86405,Meitingen,48.555683,10.850848,2017-04-07 00:00:26+02,{},2025-04-30
2981,2025-04-01 00:15:00,00006210-0037-4444-8888-acdc00006210,1.649,1.779,1.719,00006210-0037-4444-8888-acdc00006210,Beducker - Qualität günstig tanken,Beducker,Donauwörther Str.,47a,86405,Meitingen,48.555683,10.850848,2017-04-07 00:00:26+02,{},2025-04-30
2982,2025-04-01 00:30:00,00006210-0037-4444-8888-acdc00006210,1.649,1.779,1.719,00006210-0037-4444-8888-acdc00006210,Beducker - Qualität günstig tanken,Beducker,Donauwörther Str.,47a,86405,Meitingen,48.555683,10.850848,2017-04-07 00:00:26+02,{},2025-04-30
2983,2025-04-01 00:45:00,00006210-0037-4444-8888-acdc00006210,1.649,1.779,1.719,00006210-0037-4444-8888-acdc00006210,Beducker - Qualität günstig tanken,Beducker,Donauwörther Str.,47a,86405,Meitingen,48.555683,10.850848,2017-04-07 00:00:26+02,{},2025-04-30
2984,2025-04-01 01:00:00,00006210-0037-4444-8888-acdc00006210,1.649,1.779,1.719,00006210-0037-4444-8888-acdc00006210,Beducker - Qualität günstig tanken,Beducker,Donauwörther Str.,47a,86405,Meitingen,48.555683,10.850848,2017-04-07 00:00:26+02,{},2025-04-30


In [9]:
# 1) Sicherstellen, dass Datum im richtigen Format ist
df_clean["date"] = pd.to_datetime(df_clean["date"])

# 2) Letztes Beobachtungsdatum pro Station
last_seen = (
    df_clean.groupby("station_uuid", as_index=False)["date"]
    .max()
    .rename(columns={"date": "last_seen"})
)

# 3) Neuen DataFrame erstellen (nicht df_clean überschreiben!)
df_filtered = df_clean.merge(last_seen, on="station_uuid", how="left")
df_filtered["last_seen"] = pd.to_datetime(df_filtered["last_seen"])

# 4) Gleitender Aktivitäts- und Preisfilter
cutoff = pd.Timedelta(days=15)
mask_active = (df_filtered["date"] - df_filtered["last_seen"]) <= cutoff
mask_price  = df_filtered["diesel"] > 0

df_filtered = df_filtered[mask_active & mask_price].copy()

print("Datensätze nach Filter:", len(df_filtered))


Datensätze nach Filter: 43320521


In [10]:
# Replace 0 with NaN for all price columns
price_columns = ["diesel", "e5", "e10"]

# Replace 0 and negative values with NA in price columns (if they exist)
for col in price_columns:
    if col in df_filtered.columns:
        df_filtered[col] = df_filtered[col].apply(lambda x: pd.NA if x <= 0 else x)

In [11]:
# Handling opening times

def is_open(opening_json, timestamp):
    try:
        if opening_json.strip() == "{}":
            return True  # always open according to MTS-K

        data = json.loads(opening_json)
        weekday = timestamp.weekday()  # 0 = Monday, ..., 6 = Sunday
        time_str = timestamp.strftime("%H:%M")

        for block in data.get("openingTimes", []):
            if block.get("applicable_days", 0) & (1 << weekday):
                for period in block.get("periods", []):
                    if period["startp"] <= time_str < period["endp"]:
                        return True
        return False  # none of the time blocks apply
    except:
        return True  # if uncertain: treat as open

# Create a copy
df_clean_opening = df_filtered.copy()

# Price columns to set to NaN if station is closed
price_columns = [col for col in ["diesel", "e5", "e10"] if col in df_clean_opening.columns]

# Calculate opening status for each row
df_clean_opening["is_open"] = df_clean_opening.apply(
    lambda row: is_open(row["openingtimes_json"], row["date"]), axis=1
)

# Set all prices to NaN if closed
for col in price_columns:
    df_clean_opening.loc[~df_clean_opening["is_open"], col] = pd.NA


In [12]:
# Verification and exclusion of highway stations using the stations dataset
autobahn_pattern = (
    r"\b(?:"
    r"autobahn(?:\s*tankstelle)?"
    r"|autohof"
    r"|rasthof"
    r"|rast[-\s]?stätte"
    r"|raststaette"
    r"|tank\s*(?:und|&)\s*rast"
    r"|service\s*area"
    r"|an\s+der\s+a[0-9]{1,2}"
    r"|a\s*[0-9]{1,2}"
    r"|ausfahrt"
    r"|abfahrt"
    r"|bundesautobahn"
    r")\b"
)

# 2. Identify highway stations by name or street
is_highway = (
    stations_latest["name"].str.contains(autobahn_pattern, case=False, na=False) |
    stations_latest["street"].str.contains(autobahn_pattern, case=False, na=False)
)

# 3. Get UUIDs of identified highway stations
highway_ids = stations_latest[is_highway]["uuid"].unique()

# 4. Exclude these stations from df_clean_opening
df_clean_opening_excl_highway = df_clean_opening[
    ~df_clean_opening["station_uuid"].isin(highway_ids)
].copy()

# 5. Output
print("Number of detected highway stations:", len(highway_ids))
print("Rows before exclusion:", len(df_clean_opening))
print("Rows after exclusion:", len(df_clean_opening_excl_highway))

Number of detected highway stations: 452
Rows before exclusion: 43320521
Rows after exclusion: 42410097


# 3. Determining the rankings

In [7]:
# Extract fixed coordinates per station
stations_df = df_clean_opening.drop_duplicates("station_uuid")[["station_uuid", "latitude", "longitude"]]
coords_rad = np.radians(stations_df[["latitude", "longitude"]].to_numpy())

# Build BallTree with Haversine metric (Earth radius ~6371 km)
tree = BallTree(coords_rad, metric="haversine")
radius_km = 5
radius_rad = radius_km / 6371.0

# Find neighbors within 5 km radius for each station
indices = tree.query_radius(coords_rad, r=radius_rad)

# Mapping: station_uuid → list of neighbor station_uuids
uuid_list = stations_df["station_uuid"].tolist()

neighbors_dict = {
    uuid_list[i]: list(stations_df.iloc[indices[i]]["station_uuid"])
    for i in range(len(uuid_list))
}

In [8]:
# Preparation
fuels = ["diesel", "e5", "e10"]
time_column = "date"

os.makedirs("Data/rankings", exist_ok=True)

df_opening_work = df_clean_opening.copy()
df_opening_work.drop(columns=[
    "street", "house_number", "city",
    "first_active", "openingtimes_json"
], inplace=True, errors="ignore")

# Function: process a single time window completely
def process_time_window(timestamp, df_time):
    time_str = pd.to_datetime(timestamp).strftime("%Y-%m-%d_%H-%M")
    output_path = f"Data/rankings/ranking_{time_str}.csv"

    if os.path.exists(output_path):
        print(f"✓ Skipping {time_str}, file already exists.")
        return

    df_time = df_time.copy()
    df_indexed = df_time.set_index("station_uuid")

    for fuel in fuels:
        fuel_lookup = df_indexed[fuel]
        ranks = []

        for row in df_time.itertuples(index=False):
            price_value = getattr(row, fuel)
            if pd.isna(price_value):
                ranks.append(np.nan)
                continue

            uuid = row.station_uuid
            neighbor_ids = neighbors_dict.get(uuid, [])
            valid_ids = [nid for nid in neighbor_ids if nid in fuel_lookup.index]
            neighbor_prices = fuel_lookup.loc[valid_ids].dropna()
            rank = (neighbor_prices < price_value).sum() + 1
            ranks.append(rank)

        df_time[f"rank_{fuel}_5km"] = ranks

    df_time.to_csv(output_path, index=False)
    print(f"✓ Saved: {output_path}")
    del df_time, df_indexed, fuel_lookup, ranks
    gc.collect()

# Prepare time windows
time_windows = list(df_opening_work.groupby(time_column))

# Parallel processing over time windows
Parallel(n_jobs=-1, backend="loky")(
    delayed(process_time_window)(timestamp, df_time)
    for timestamp, df_time in tqdm(time_windows, desc="Time windows")
)


Time windows:   0%|          | 0/2872 [00:00<?, ?it/s]

✓ Saved: Data/rankings/ranking_2025-04-01_02-45.csv
✓ Saved: Data/rankings/ranking_2025-04-01_05-00.csv
✓ Saved: Data/rankings/ranking_2025-04-01_08-00.csv
✓ Saved: Data/rankings/ranking_2025-04-01_11-00.csv
✓ Saved: Data/rankings/ranking_2025-04-01_14-00.csv
✓ Saved: Data/rankings/ranking_2025-04-01_17-00.csv
✓ Saved: Data/rankings/ranking_2025-04-01_20-00.csv
✓ Saved: Data/rankings/ranking_2025-04-01_22-30.csv
✓ Saved: Data/rankings/ranking_2025-04-02_00-30.csv
✓ Saved: Data/rankings/ranking_2025-04-02_03-30.csv
✓ Saved: Data/rankings/ranking_2025-04-02_06-30.csv
✓ Saved: Data/rankings/ranking_2025-04-02_09-45.csv
✓ Saved: Data/rankings/ranking_2025-04-02_12-30.csv
✓ Saved: Data/rankings/ranking_2025-04-02_15-30.csv
✓ Saved: Data/rankings/ranking_2025-04-02_18-30.csv
✓ Saved: Data/rankings/ranking_2025-04-02_21-30.csv
✓ Saved: Data/rankings/ranking_2025-04-03_01-30.csv
✓ Saved: Data/rankings/ranking_2025-04-03_04-15.csv
✓ Saved: Data/rankings/ranking_2025-04-03_07-15.csv
✓ Saved: Dat

✓ Saved: Data/rankings/ranking_2025-04-01_00-45.csv
✓ Saved: Data/rankings/ranking_2025-04-01_03-15.csv
✓ Saved: Data/rankings/ranking_2025-04-01_06-15.csv
✓ Saved: Data/rankings/ranking_2025-04-01_09-00.csv
✓ Saved: Data/rankings/ranking_2025-04-01_12-00.csv
✓ Saved: Data/rankings/ranking_2025-04-01_15-30.csv
✓ Saved: Data/rankings/ranking_2025-04-01_18-30.csv
✓ Saved: Data/rankings/ranking_2025-04-01_21-15.csv
✓ Saved: Data/rankings/ranking_2025-04-02_02-00.csv
✓ Saved: Data/rankings/ranking_2025-04-02_05-00.csv
✓ Saved: Data/rankings/ranking_2025-04-02_08-00.csv
✓ Saved: Data/rankings/ranking_2025-04-02_11-15.csv
✓ Saved: Data/rankings/ranking_2025-04-02_14-15.csv
✓ Saved: Data/rankings/ranking_2025-04-02_17-15.csv
✓ Saved: Data/rankings/ranking_2025-04-02_20-15.csv
✓ Saved: Data/rankings/ranking_2025-04-02_23-15.csv
✓ Saved: Data/rankings/ranking_2025-04-03_02-00.csv
✓ Saved: Data/rankings/ranking_2025-04-03_04-45.csv
✓ Saved: Data/rankings/ranking_2025-04-03_07-45.csv
✓ Saved: Dat

✓ Saved: Data/rankings/ranking_2025-04-01_02-15.csv
✓ Saved: Data/rankings/ranking_2025-04-01_05-30.csv
✓ Saved: Data/rankings/ranking_2025-04-01_08-30.csv
✓ Saved: Data/rankings/ranking_2025-04-01_11-30.csv
✓ Saved: Data/rankings/ranking_2025-04-01_14-30.csv
✓ Saved: Data/rankings/ranking_2025-04-01_17-30.csv
✓ Saved: Data/rankings/ranking_2025-04-01_20-30.csv
✓ Saved: Data/rankings/ranking_2025-04-01_23-30.csv
✓ Saved: Data/rankings/ranking_2025-04-02_01-30.csv
✓ Saved: Data/rankings/ranking_2025-04-02_04-30.csv
✓ Saved: Data/rankings/ranking_2025-04-02_07-30.csv
✓ Saved: Data/rankings/ranking_2025-04-02_10-15.csv
✓ Saved: Data/rankings/ranking_2025-04-02_13-30.csv
✓ Saved: Data/rankings/ranking_2025-04-02_16-15.csv
✓ Saved: Data/rankings/ranking_2025-04-02_19-15.csv
✓ Saved: Data/rankings/ranking_2025-04-02_22-15.csv
✓ Saved: Data/rankings/ranking_2025-04-03_00-00.csv
✓ Saved: Data/rankings/ranking_2025-04-03_03-00.csv
✓ Saved: Data/rankings/ranking_2025-04-03_06-00.csv
✓ Saved: Dat

✓ Saved: Data/rankings/ranking_2025-04-01_01-15.csv
✓ Saved: Data/rankings/ranking_2025-04-01_04-45.csv
✓ Saved: Data/rankings/ranking_2025-04-01_07-45.csv
✓ Saved: Data/rankings/ranking_2025-04-01_10-30.csv
✓ Saved: Data/rankings/ranking_2025-04-01_13-30.csv
✓ Saved: Data/rankings/ranking_2025-04-01_16-30.csv
✓ Saved: Data/rankings/ranking_2025-04-01_19-30.csv
✓ Saved: Data/rankings/ranking_2025-04-01_22-45.csv
✓ Saved: Data/rankings/ranking_2025-04-02_01-00.csv
✓ Saved: Data/rankings/ranking_2025-04-02_04-00.csv
✓ Saved: Data/rankings/ranking_2025-04-02_07-00.csv
✓ Saved: Data/rankings/ranking_2025-04-02_10-00.csv
✓ Saved: Data/rankings/ranking_2025-04-02_13-00.csv
✓ Saved: Data/rankings/ranking_2025-04-02_16-00.csv
✓ Saved: Data/rankings/ranking_2025-04-02_19-00.csv
✓ Saved: Data/rankings/ranking_2025-04-02_22-00.csv
✓ Saved: Data/rankings/ranking_2025-04-02_23-45.csv
✓ Saved: Data/rankings/ranking_2025-04-03_02-45.csv
✓ Saved: Data/rankings/ranking_2025-04-03_05-45.csv
✓ Saved: Dat

✓ Saved: Data/rankings/ranking_2025-04-01_02-30.csv
✓ Saved: Data/rankings/ranking_2025-04-01_05-15.csv
✓ Saved: Data/rankings/ranking_2025-04-01_08-15.csv
✓ Saved: Data/rankings/ranking_2025-04-01_11-15.csv
✓ Saved: Data/rankings/ranking_2025-04-01_14-15.csv
✓ Saved: Data/rankings/ranking_2025-04-01_17-15.csv
✓ Saved: Data/rankings/ranking_2025-04-01_20-15.csv
✓ Saved: Data/rankings/ranking_2025-04-01_23-15.csv
✓ Saved: Data/rankings/ranking_2025-04-02_01-15.csv
✓ Saved: Data/rankings/ranking_2025-04-02_04-15.csv
✓ Saved: Data/rankings/ranking_2025-04-02_07-15.csv
✓ Saved: Data/rankings/ranking_2025-04-02_10-30.csv
✓ Saved: Data/rankings/ranking_2025-04-02_13-15.csv
✓ Saved: Data/rankings/ranking_2025-04-02_16-30.csv
✓ Saved: Data/rankings/ranking_2025-04-02_19-30.csv
✓ Saved: Data/rankings/ranking_2025-04-02_22-45.csv
✓ Saved: Data/rankings/ranking_2025-04-03_00-30.csv
✓ Saved: Data/rankings/ranking_2025-04-03_03-30.csv
✓ Saved: Data/rankings/ranking_2025-04-03_06-30.csv
✓ Saved: Dat

✓ Saved: Data/rankings/ranking_2025-04-01_00-30.csv
✓ Saved: Data/rankings/ranking_2025-04-01_05-45.csv
✓ Saved: Data/rankings/ranking_2025-04-01_08-45.csv
✓ Saved: Data/rankings/ranking_2025-04-01_11-45.csv
✓ Saved: Data/rankings/ranking_2025-04-01_14-45.csv
✓ Saved: Data/rankings/ranking_2025-04-01_17-45.csv
✓ Saved: Data/rankings/ranking_2025-04-01_20-45.csv
✓ Saved: Data/rankings/ranking_2025-04-01_23-45.csv
✓ Saved: Data/rankings/ranking_2025-04-02_02-45.csv
✓ Saved: Data/rankings/ranking_2025-04-02_05-45.csv
✓ Saved: Data/rankings/ranking_2025-04-02_08-45.csv
✓ Saved: Data/rankings/ranking_2025-04-02_11-45.csv
✓ Saved: Data/rankings/ranking_2025-04-02_14-45.csv
✓ Saved: Data/rankings/ranking_2025-04-02_17-45.csv
✓ Saved: Data/rankings/ranking_2025-04-02_20-45.csv
✓ Saved: Data/rankings/ranking_2025-04-03_01-45.csv
✓ Saved: Data/rankings/ranking_2025-04-03_05-00.csv
✓ Saved: Data/rankings/ranking_2025-04-03_08-00.csv
✓ Saved: Data/rankings/ranking_2025-04-03_11-15.csv
✓ Saved: Dat

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [9]:
# Combine all files
all_parts = glob.glob("Data/rankings/ranking_*.csv")
df_final = pd.concat([pd.read_csv(f) for f in all_parts], ignore_index=True)
df_final.to_csv("Data/Temp/df_clean_opening_ranked.csv", index=False)

✓ Saved: Data/rankings/ranking_2025-04-19_18-45.csv
✓ Saved: Data/rankings/ranking_2025-04-19_21-45.csv
✓ Saved: Data/rankings/ranking_2025-04-20_02-30.csv
✓ Saved: Data/rankings/ranking_2025-04-20_05-30.csv
✓ Saved: Data/rankings/ranking_2025-04-20_08-30.csv
✓ Saved: Data/rankings/ranking_2025-04-20_11-30.csv
✓ Saved: Data/rankings/ranking_2025-04-20_14-30.csv
✓ Saved: Data/rankings/ranking_2025-04-20_17-30.csv
✓ Saved: Data/rankings/ranking_2025-04-20_20-30.csv
✓ Saved: Data/rankings/ranking_2025-04-20_23-30.csv
✓ Saved: Data/rankings/ranking_2025-04-21_02-30.csv
✓ Saved: Data/rankings/ranking_2025-04-21_05-30.csv
✓ Saved: Data/rankings/ranking_2025-04-21_08-30.csv
✓ Saved: Data/rankings/ranking_2025-04-21_11-30.csv
✓ Saved: Data/rankings/ranking_2025-04-21_14-30.csv
✓ Saved: Data/rankings/ranking_2025-04-21_17-30.csv
✓ Saved: Data/rankings/ranking_2025-04-21_20-30.csv
✓ Saved: Data/rankings/ranking_2025-04-21_23-15.csv
✓ Saved: Data/rankings/ranking_2025-04-22_02-15.csv
✓ Saved: Dat

✓ Saved: Data/rankings/ranking_2025-04-19_23-15.csv
✓ Saved: Data/rankings/ranking_2025-04-20_01-45.csv
✓ Saved: Data/rankings/ranking_2025-04-20_05-00.csv
✓ Saved: Data/rankings/ranking_2025-04-20_08-00.csv
✓ Saved: Data/rankings/ranking_2025-04-20_11-00.csv
✓ Saved: Data/rankings/ranking_2025-04-20_14-00.csv
✓ Saved: Data/rankings/ranking_2025-04-20_17-00.csv
✓ Saved: Data/rankings/ranking_2025-04-20_20-00.csv
✓ Saved: Data/rankings/ranking_2025-04-20_23-00.csv
✓ Saved: Data/rankings/ranking_2025-04-21_02-00.csv
✓ Saved: Data/rankings/ranking_2025-04-21_05-00.csv
✓ Saved: Data/rankings/ranking_2025-04-21_08-00.csv
✓ Saved: Data/rankings/ranking_2025-04-21_11-00.csv
✓ Saved: Data/rankings/ranking_2025-04-21_14-00.csv
✓ Saved: Data/rankings/ranking_2025-04-21_17-00.csv
✓ Saved: Data/rankings/ranking_2025-04-21_20-00.csv
✓ Saved: Data/rankings/ranking_2025-04-21_23-00.csv
✓ Saved: Data/rankings/ranking_2025-04-22_02-00.csv
✓ Saved: Data/rankings/ranking_2025-04-22_04-45.csv
✓ Saved: Dat

✓ Saved: Data/rankings/ranking_2025-04-19_22-45.csv
✓ Saved: Data/rankings/ranking_2025-04-20_02-00.csv
✓ Saved: Data/rankings/ranking_2025-04-20_04-45.csv
✓ Saved: Data/rankings/ranking_2025-04-20_07-45.csv
✓ Saved: Data/rankings/ranking_2025-04-20_10-45.csv
✓ Saved: Data/rankings/ranking_2025-04-20_13-45.csv
✓ Saved: Data/rankings/ranking_2025-04-20_16-45.csv
✓ Saved: Data/rankings/ranking_2025-04-20_19-30.csv
✓ Saved: Data/rankings/ranking_2025-04-20_22-30.csv
✓ Saved: Data/rankings/ranking_2025-04-21_01-15.csv
✓ Saved: Data/rankings/ranking_2025-04-21_04-15.csv
✓ Saved: Data/rankings/ranking_2025-04-21_07-15.csv
✓ Saved: Data/rankings/ranking_2025-04-21_10-15.csv
✓ Saved: Data/rankings/ranking_2025-04-21_13-15.csv
✓ Saved: Data/rankings/ranking_2025-04-21_16-00.csv
✓ Saved: Data/rankings/ranking_2025-04-21_19-00.csv
✓ Saved: Data/rankings/ranking_2025-04-21_22-00.csv
✓ Saved: Data/rankings/ranking_2025-04-22_00-30.csv
✓ Saved: Data/rankings/ranking_2025-04-22_03-30.csv
✓ Saved: Dat

✓ Saved: Data/rankings/ranking_2025-04-19_23-30.csv
✓ Saved: Data/rankings/ranking_2025-04-20_02-15.csv
✓ Saved: Data/rankings/ranking_2025-04-20_05-15.csv
✓ Saved: Data/rankings/ranking_2025-04-20_08-15.csv
✓ Saved: Data/rankings/ranking_2025-04-20_11-15.csv
✓ Saved: Data/rankings/ranking_2025-04-20_14-15.csv
✓ Saved: Data/rankings/ranking_2025-04-20_17-15.csv
✓ Saved: Data/rankings/ranking_2025-04-20_20-15.csv
✓ Saved: Data/rankings/ranking_2025-04-20_23-15.csv
✓ Saved: Data/rankings/ranking_2025-04-21_02-15.csv
✓ Saved: Data/rankings/ranking_2025-04-21_05-15.csv
✓ Saved: Data/rankings/ranking_2025-04-21_08-15.csv
✓ Saved: Data/rankings/ranking_2025-04-21_11-15.csv
✓ Saved: Data/rankings/ranking_2025-04-21_14-15.csv
✓ Saved: Data/rankings/ranking_2025-04-21_17-15.csv
✓ Saved: Data/rankings/ranking_2025-04-21_20-15.csv
✓ Saved: Data/rankings/ranking_2025-04-21_23-30.csv
✓ Saved: Data/rankings/ranking_2025-04-22_02-30.csv
✓ Saved: Data/rankings/ranking_2025-04-22_05-30.csv
✓ Saved: Dat

✓ Saved: Data/rankings/ranking_2025-04-19_18-30.csv
✓ Saved: Data/rankings/ranking_2025-04-19_21-30.csv
✓ Saved: Data/rankings/ranking_2025-04-20_01-00.csv
✓ Saved: Data/rankings/ranking_2025-04-20_04-00.csv
✓ Saved: Data/rankings/ranking_2025-04-20_07-00.csv
✓ Saved: Data/rankings/ranking_2025-04-20_10-00.csv
✓ Saved: Data/rankings/ranking_2025-04-20_13-00.csv
✓ Saved: Data/rankings/ranking_2025-04-20_16-00.csv
✓ Saved: Data/rankings/ranking_2025-04-20_19-00.csv
✓ Saved: Data/rankings/ranking_2025-04-20_21-45.csv
✓ Saved: Data/rankings/ranking_2025-04-21_01-45.csv
✓ Saved: Data/rankings/ranking_2025-04-21_04-45.csv
✓ Saved: Data/rankings/ranking_2025-04-21_07-45.csv
✓ Saved: Data/rankings/ranking_2025-04-21_10-45.csv
✓ Saved: Data/rankings/ranking_2025-04-21_13-45.csv
✓ Saved: Data/rankings/ranking_2025-04-21_16-45.csv
✓ Saved: Data/rankings/ranking_2025-04-21_19-45.csv
✓ Saved: Data/rankings/ranking_2025-04-21_22-45.csv
✓ Saved: Data/rankings/ranking_2025-04-22_01-00.csv
✓ Saved: Dat

✓ Saved: Data/rankings/ranking_2025-04-20_02-45.csv
✓ Saved: Data/rankings/ranking_2025-04-20_06-00.csv
✓ Saved: Data/rankings/ranking_2025-04-20_09-00.csv
✓ Saved: Data/rankings/ranking_2025-04-20_12-00.csv
✓ Saved: Data/rankings/ranking_2025-04-20_15-00.csv
✓ Saved: Data/rankings/ranking_2025-04-20_18-00.csv
✓ Saved: Data/rankings/ranking_2025-04-20_21-00.csv
✓ Saved: Data/rankings/ranking_2025-04-21_00-00.csv
✓ Saved: Data/rankings/ranking_2025-04-21_03-00.csv
✓ Saved: Data/rankings/ranking_2025-04-21_05-45.csv
✓ Saved: Data/rankings/ranking_2025-04-21_08-45.csv
✓ Saved: Data/rankings/ranking_2025-04-21_11-45.csv
✓ Saved: Data/rankings/ranking_2025-04-21_14-45.csv
✓ Saved: Data/rankings/ranking_2025-04-21_17-45.csv
✓ Saved: Data/rankings/ranking_2025-04-21_20-45.csv
✓ Saved: Data/rankings/ranking_2025-04-21_23-45.csv
✓ Saved: Data/rankings/ranking_2025-04-22_02-45.csv
✓ Saved: Data/rankings/ranking_2025-04-22_05-45.csv
✓ Saved: Data/rankings/ranking_2025-04-22_08-45.csv
✓ Saved: Dat

OSError: [Errno 122] Disk quota exceeded

# 4. Generate Features

In [2]:
# Load data (restore from file)
df_final = pd.read_csv("Data/Temp/df_clean_opening_ranked.csv")


/var/tmp/pbs.14231814.hpc-batch/ipykernel_34060/678013521.py:2: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_final = pd.read_csv("Data/Temp/temporary.csv")


In [13]:
# Generate lagged rank features (for lead prediction) for each fuel type

# First, sort by station and date to ensure correct shifting
df = df_final.sort_values(['station_uuid', 'date'])

# Create future rank features for Diesel (t+1 to t+8)
df['rank_diesel_5km+1'] = df.groupby('station_uuid')['rank_diesel_5km'].shift(-1)
df['rank_diesel_5km+2'] = df.groupby('station_uuid')['rank_diesel_5km'].shift(-2)
df['rank_diesel_5km+3'] = df.groupby('station_uuid')['rank_diesel_5km'].shift(-3)
df['rank_diesel_5km+4'] = df.groupby('station_uuid')['rank_diesel_5km'].shift(-4)
df['rank_diesel_5km+5'] = df.groupby('station_uuid')['rank_diesel_5km'].shift(-5)
df['rank_diesel_5km+6'] = df.groupby('station_uuid')['rank_diesel_5km'].shift(-6)
df['rank_diesel_5km+7'] = df.groupby('station_uuid')['rank_diesel_5km'].shift(-7)
df['rank_diesel_5km+8'] = df.groupby('station_uuid')['rank_diesel_5km'].shift(-8)

# Create future rank features for E5 (t+1 to t+8)
df['rank_e5_5km+1'] = df.groupby('station_uuid')['rank_e5_5km'].shift(-1)
df['rank_e5_5km+2'] = df.groupby('station_uuid')['rank_e5_5km'].shift(-2)
df['rank_e5_5km+3'] = df.groupby('station_uuid')['rank_e5_5km'].shift(-3)
df['rank_e5_5km+4'] = df.groupby('station_uuid')['rank_e5_5km'].shift(-4)
df['rank_e5_5km+5'] = df.groupby('station_uuid')['rank_e5_5km'].shift(-5)
df['rank_e5_5km+6'] = df.groupby('station_uuid')['rank_e5_5km'].shift(-6)
df['rank_e5_5km+7'] = df.groupby('station_uuid')['rank_e5_5km'].shift(-7)
df['rank_e5_5km+8'] = df.groupby('station_uuid')['rank_e5_5km'].shift(-8)

# Create future rank features for E10 (t+1 to t+8)
df['rank_e10_5km+1'] = df.groupby('station_uuid')['rank_e10_5km'].shift(-1)
df['rank_e10_5km+2'] = df.groupby('station_uuid')['rank_e10_5km'].shift(-2)
df['rank_e10_5km+3'] = df.groupby('station_uuid')['rank_e10_5km'].shift(-3)
df['rank_e10_5km+4'] = df.groupby('station_uuid')['rank_e10_5km'].shift(-4)
df['rank_e10_5km+5'] = df.groupby('station_uuid')['rank_e10_5km'].shift(-5)
df['rank_e10_5km+6'] = df.groupby('station_uuid')['rank_e10_5km'].shift(-6)
df['rank_e10_5km+7'] = df.groupby('station_uuid')['rank_e10_5km'].shift(-7)
df['rank_e10_5km+8'] = df.groupby('station_uuid')['rank_e10_5km'].shift(-8)

In [11]:
# Display the first few rows of the dataframe for inspection
df.head()

,date,station_uuid,diesel,e5,e10,uuid,name,brand,post_code,latitude,...,rank_e5_5km+7,rank_e5_5km+8,rank_e10_5km+1,rank_e10_5km+2,rank_e10_5km+3,rank_e10_5km+4,rank_e10_5km+5,rank_e10_5km+6,rank_e10_5km+7,rank_e10_5km+8
0,2025-04-01 00:00:00,00006210-0037-4444-8888-acdc00006210,1.649,1.779,1.719,00006210-0037-4444-8888-acdc00006210,Beducker - Qualität günstig tanken,Beducker,86405,48.555683,...,3.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
45246,2025-04-01 00:15:00,00006210-0037-4444-8888-acdc00006210,1.649,1.779,1.719,00006210-0037-4444-8888-acdc00006210,Beducker - Qualität günstig tanken,Beducker,86405,48.555683,...,3.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
165902,2025-04-01 00:30:00,00006210-0037-4444-8888-acdc00006210,1.649,1.779,1.719,00006210-0037-4444-8888-acdc00006210,Beducker - Qualität günstig tanken,Beducker,86405,48.555683,...,3.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
15082,2025-04-01 00:45:00,00006210-0037-4444-8888-acdc00006210,1.649,1.779,1.719,00006210-0037-4444-8888-acdc00006210,Beducker - Qualität günstig tanken,Beducker,86405,48.555683,...,3.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
30164,2025-04-01 01:00:00,00006210-0037-4444-8888-acdc00006210,1.649,1.779,1.719,00006210-0037-4444-8888-acdc00006210,Beducker - Qualität günstig tanken,Beducker,86405,48.555683,...,3.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0


In [12]:
# Save the final dataframe to CSV
df.to_csv('Data/Temp/final.csv', index=False)